# 0.Import of packages 

In [ ]:
import muscle_analysis as ma
import tkinter as tk
import tkinter.filedialog as fd
import json
import scipy.io

root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

# 1.Uploading of transformations and setting of the parameters


In [ ]:
file_path = fd.askopenfilename(title = "Choose the position list")
path_smFRET = fd.askdirectory(title = "Choose the smFRET folder")
pos_file = open(file_path)
data = json.load(pos_file)
POS = data['POSITIONS']
tr_G2R, tr_R2G = ma.load_data.poly_transf()
apriori_tr_original, apriori_tr, apriori_tr_inv = ma.load_data.apriori_transf(ma.x_border,ma.y_border) #uploading  transformations

ALEX = True #if imaging was performed using ALEX, otherwise False
read2 = True #if paired-end sequencing was  used, otherwise False 
current_tile = 1101 # number of the tile from the fastq file 


# 2.Analysis of the FASTQ file

In [ ]:
x_coord, y_coord, sequence = ma.muscle_sequencing.create_FASTQ_image(current_tile, read2)
#create the FASTQ file, if read2 = True, then sequence is an 2D array

# 3.Creating the combined image 

In [ ]:
ma.pks_from_img.combined_image(path_smFRET, POS, ALEX, tr_R2G, apriori_tr_original) #create combined image

# 4. Alignment

In [ ]:
rb_rad, r = ma.align.alignment(path_smFRET, POS, ALEX, tr_R2G, apriori_tr, x_coord, y_coord, sequence, read2) #main cycle

# 5. Trace extraction 

In [ ]:
ma.traces.extract_traces(path_smFRET, read2, rb_rad, r) #extract traces

In [5]:
help(ma.muscle_sequencing.create_FASTQ_image)

Help on function create_FASTQ_image in module muscle_analysis.muscle_sequencing:

create_FASTQ_image(current_tile, read2)
    This function is used for creating the image based on FASTQ file. 
    
    Args:
        current tile(integer): number of the current tile
        read2(boolean):  in case of paired-end sequencing, this flag should be True
    Returns:
       Creates the fastq image 
       x_coord - x coordinates from the Fastq file 
       y_coord - y coordinates from the Fastq file
       sequence - list of selected sequences. if read2 =True, it's a 2D array



In [4]:
help(ma.pks_from_img.combined_image)

Help on function combined_image in module muscle_analysis.pks_from_img:

combined_image(path_smFRET, POS, ALEX, tr_R2G, apriori_tr_original)
    Creating the combined image from the sm_FRET FOVs
    
    Args:
        path_smFRET: path to the folder with smFRET movies.
        POS: position list.
        ALEX (bool): was the data collected using alternating laser excitation (ALEX)?.
        tr_R2G: the polynomial reverse transformation for mapping the Cy3 and Cy5 channels
        apriori_tr_original: original apriori transformation from smFRET to sequencing coordinates
    Returns:
        The function creates an image that combines all smFRET FOVs in order to perform preliminary alignment with FASTQ tiles



In [3]:
help (ma.align.alignment)

Help on function alignment in module muscle_analysis.align:

alignment(path_smFRET, POS, ALEX, tr_R2G, apriori_tr, x_coord, y_coord, sequence, read2)
    The function is used for the alingment between molecules from smFRET imaging and clusters from sequencing.
    
    Args:
        path_smFRET (string): The absolute path of the folder with smFRET movies.
        POS: the information about FOV positions. 
        ALEX (boolean): Was imaging performed with alternating laser excitation (ALEX)?.
        tr_R2G : forward transformation mapping Cy3 and Cy5 channels
        apriori_tr: a preliminary transformation from smFRET to sequencing coordinates
        x_coord: list of the x coordinates from the FASTQ file
        y_coord: list of the y coordinates from the FASTQ file
        sequence: list of the sequences from the FASTQ file, in case of paired-end sequencing, it is 2D array
        read2: in case of paired-end sequencing, this flag should be True
    Returns:
         It returns the

In [2]:
help(ma.traces.extract_traces)

Help on function extract_traces in module muscle_analysis.traces:

extract_traces(path_smFRET, read2, rb_rad, r)
    This function is creating the .mat files containing matched smFRET traces and sequences for molecules from FOVs that were aligned in the previous step
    
    Args:
        path_smFRET - path to the folder containing the smFRET movies. 
        read2 - flag for paired-end sequencing
    Returns:
        rb_rad - rolling ball radius
        r - Half-width of the molecule aperture for trace extraction, i.e. for r = 3 it is -3:3
      
        "time": time information from smFRET trace 
        "Cy3": Cy3 intensity 
        "Cy5": Cy5 intensity 
        "Seq": sequence based on the read1,
        "Seq_2": sequence based on read2 (if not paired-end, it's an empty array),
        "Dist": distance betweeen a cluster and a corresponding single molecule,
        "x": x coordinates of single molecules from the Cy5 channel,
        "y": y coordinates of single molecules from the 